In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import Model, backend as K

class NeuralStyleTransfer:
    def __init__(self, content_image_path, style_image_path, learning_rate=0.01):
        self.content_image = self.load_and_preprocess_image(content_image_path)
        self.style_image = self.load_and_preprocess_image(style_image_path)
        self.generated_image = tf.Variable(self.content_image, dtype=tf.float32)
        self.learning_rate = learning_rate
        self.model = self.build_model()

    def load_and_preprocess_image(self, path):
        img = load_img(path, target_size=(224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = vgg19.preprocess_input(img)
        return img

    def build_model(self):
        base_model = vgg19.VGG19(weights='imagenet', include_top=False)
        outputs = [base_model.get_layer(name).output for name in ['block5_conv2', 'block4_conv2']]
        return Model(inputs=base_model.input, outputs=outputs)

    def compute_loss(self):
        style_features = self.model(self.style_image)
        content_features = self.model(self.content_image)
        generated_features = self.model(self.generated_image)

        content_loss = K.sum(K.square(content_features[0] - generated_features[0]))
        style_loss = K.sum(K.square(style_features[0] - generated_features[1]))  # Simplified for one layer

        total_loss = content_loss + 0.01 * style_loss  # Weighting factor
        return total_loss

    def optimize(self, iterations=100):
        optimizer = tf.optimizers.Adam(learning_rate=self.learning_rate)
        for i in range(iterations):
            with tf.GradientTape() as tape:
                loss = self.compute_loss()
            grads = tape.gradient(loss, self.generated_image)
            optimizer.apply_gradients([(grads, self.generated_image)])
            if i % 10 == 0:
                print(f'Iteration {i}, Loss: {loss.numpy()}')
        return self.generated_image

nst = NeuralStyleTransfer('content.jpg', 'style.jpg')
output_image = nst.optimize()
output_image = tf.clip_by_value(output_image, 0, 255).numpy().astype('uint8')[0]
plt.imshow(output_image)
plt.axis('off')
plt.show()